This notebook is where data is read in from csv files and put into data frames.  There is a little bit of data clean up and merging of dataframes.  Data is saved into pickled files.

In [1]:
import pandas as pd
import pickle
import numpy as np
import psycopg2 as pg
from sqlalchemy import create_engine
import io

## Murder Data

In [2]:
# Reads the murders csv file into a pandas dataframe
df = pd.read_csv('SHR76_17.csv')

In [3]:
df.head()

,ID,CNTYFIPS,Ori,State,Agency,Agentype,Source,Solved,Year,Month,...,OffEthnic,Weapon,Relationship,Circumstance,Subcircum,VicCount,OffCount,FileDate,fstate,MSA
0,197601001AKASP00,"Juneau, AK",AKASP00,Alaska,State Troopers,Primary state LE,FBI,Yes,1976,January,...,Unknown or not reported,Knife or cutting instrument,Husband,Other arguments,NaN,0,0,30180.0,Alaska,Rural Alaska
1,197601001AL00102,"Jefferson, AL",AL00102,Alabama,Birmingham,Municipal police,FBI,Yes,1976,January,...,Unknown or not reported,Shotgun,Acquaintance,Felon killed by private citizen,Felon killed in commission of a crime,0,0,30180.0,Alabama,"Birmingham-Hoover, AL"
2,197601001AL00104,"Jefferson, AL",AL00104,Alabama,Fairfield,Municipal police,FBI,Yes,1976,January,...,Unknown or not reported,Shotgun,Wife,Other,NaN,0,0,30180.0,Alabama,"Birmingham-Hoover, AL"
3,197601001AL00106,"Jefferson, AL",AL00106,Alabama,Leeds,Municipal police,FBI,Yes,1976,January,...,Unknown or not reported,Knife or cutting instrument,Brother,Other arguments,NaN,0,0,30180.0,Alabama,"Birmingham-Hoover, AL"
4,197601001AL00201,"Mobile, AL",AL00201,Alabama,Mobile,Municipal police,FBI,Yes,1976,January,...,Unknown or not reported,Strangulation - hanging,Acquaintance,Circumstances undetermined,NaN,0,0,30180.0,Alabama,"Mobile, AL"


### Modify and save Murder Data using SQL

In [11]:
# Make a sql database from the above dataframe
engine = create_engine('postgresql+psycopg2://Julia:@127.0.0.1', pool_recycle=3600);
postgreSQLConnection = engine.connect();
postgreSQLTable = "murders";

frame = df.to_sql(postgreSQLTable, postgreSQLConnection, if_exists='fail');


In [12]:
# Make sure I successfully created the table
print(engine.table_names())

['murders']


In [6]:
con = engine.connect()

In [20]:
# Sets Solved to 1 if Solves is yes
query = """ UPDATE murders SET "Solved" = 1 WHERE "Solved" = 'Yes'

        """
con.execute(query);

In [21]:
# Sets Solved to 0 if Solves is no
query = """ UPDATE murders SET "Solved" = 0 WHERE "Solved" = 'No'

        """
con.execute(query);

In [22]:
# Saves a query to a database so we can look at the first 5 rows of our table
query = """SELECT * FROM murders;"""

df_query = pd.read_sql(query, con)
df_query.head()

,index,ID,CNTYFIPS,Ori,State,Agency,Agentype,Source,Solved,Year,...,OffEthnic,Weapon,Relationship,Circumstance,Subcircum,VicCount,OffCount,FileDate,fstate,MSA
0,37,197601001AR06002,"Pulaski, AR",AR06002,Arkansas,Little Rock,Municipal police,FBI,0,1976,...,Unknown or not reported,"Handgun - pistol, revolver, etc",Relationship not determined,Circumstances undetermined,None,0,0,30180.0,Arkansas,"Little Rock-North Little Rock, AR"
1,41,197601001AZ00700,"Maricopa, AZ",AZ00700,Arizona,Maricopa County,Sheriff,FBI,0,1976,...,Unknown or not reported,Knife or cutting instrument,Relationship not determined,Circumstances undetermined,None,0,0,30180.0,Arizona,"Phoenix-Mesa-Scottsdale, AZ"
2,48,197601001AZ01307,"Yavapai, AZ",AZ01307,Arizona,Prescott,Municipal police,FBI,0,1976,...,Unknown or not reported,Fire,Relationship not determined,Arson,None,0,0,30180.0,Arizona,"Prescott, AZ"
3,54,197601001CA00701,"Contra Costa, CA",CA00701,California,Antioch,Municipal police,FBI,0,1976,...,Unknown or not reported,"Personal weapons, includes beating",Daughter,Other,None,0,0,30180.0,California,"San Francisco-Oakland-Fremont, CA"
4,60,197601001CA01200,"Humboldt, CA",CA01200,California,Humboldt County,Sheriff,FBI,0,1976,...,Unknown or not reported,"Personal weapons, includes beating",Relationship not determined,Rape,None,0,0,30180.0,California,Rural California


In [ ]:
# Save SQL data
with open('murder_data.pickle', 'wb') as to_write:
    pickle.dump(df_query, to_write)

### Modify dataframe without using SQL

In [4]:
df['Solved_nums'] = df['Solved'].replace(['Yes', 'No'],[1,0])

In [8]:
with open('murder_data.pickle', 'wb') as to_write:
    pickle.dump(df, to_write)

## Census Data

In [17]:
# Put CSV data into dataframes
census_df = pd.read_csv('csv_pus/ss10pusa.csv')
census_df_2 = pd.read_csv('csv_pus/ss10pusb.csv')
census_df_3 = pd.read_csv('csv_pus/ss10pusc.csv')
census_df_4 = pd.read_csv('csv_pus/ss10pusd.csv')

/Users/Julia/opt/anaconda3/envs/metis/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (219,221,223,225) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [27]:
list(census_df.columns)

['serialno',
 'RT',
 'SPORDER',
 'PUMA',
 'ST',
 'ADJINC',
 'PWGTP',
 'AGEP',
 'CIT',
 'COW',
 'ENG',
 'FER',
 'GCL',
 'GCM',
 'GCR',
 'INTP',
 'JWMNP',
 'JWRIP',
 'JWTR',
 'LANX',
 'MAR',
 'MIG',
 'MIL',
 'MLPA',
 'MLPB',
 'MLPC',
 'MLPD',
 'MLPE',
 'MLPF',
 'MLPG',
 'MLPH',
 'MLPI',
 'MLPJ',
 'MLPK',
 'NWAB',
 'NWAV',
 'NWLA',
 'NWLK',
 'NWRE',
 'OIP',
 'PAP',
 'RELP',
 'RETP',
 'SCH',
 'SCHG',
 'SCHL',
 'SEMP',
 'SEX',
 'SSIP',
 'SSP',
 'WAGP',
 'WKHP',
 'WKL',
 'WKW',
 'YOEP',
 'ANC',
 'ANC1P',
 'ANC2P',
 'DECADE',
 'DRIVESP',
 'ESP',
 'ESR',
 'HISP',
 'JWAP',
 'JWDP',
 'LANP',
 'MIGPUMA',
 'MIGSP',
 'MSP',
 'NATIVITY',
 'NOP',
 'OC',
 'PAOC',
 'PERNP',
 'PINCP',
 'POBP',
 'POVPIP',
 'POWPUMA',
 'POWSP',
 'QTRBIR',
 'RAC1P',
 'RAC2P',
 'RAC3P',
 'RACAIAN',
 'RACASN',
 'RACBLK',
 'RACNHPI',
 'RACNUM',
 'RACSOR',
 'RACWHT',
 'RC',
 'SFN',
 'SFR',
 'VPS',
 'WAOB',
 'FAGEP',
 'FANCP',
 'FCITP',
 'FCOWP',
 'FENGP',
 'FESRP',
 'FFERP',
 'FGCLP',
 'FGCMP',
 'FGCRP',
 'FHISP',
 'FINDP',
 '

In [36]:
# Select Columns that will be used
kept_columns = ['PUMA', 'ST', 'WAGP', 'AGEP', 'SCHL', 'RAC1P']

In [37]:
# Selects data in desired columns and put into dataframes
df_smaller = census_df[kept_columns]
df_smaller2 = census_df_2[kept_columns]
df_smaller3 = census_df_3[kept_columns]
df_smaller4 = census_df_4[kept_columns]

In [86]:
# Concatenates dataframes into one dataframe
census_final = pd.concat([df_smaller, df_smaller2, df_smaller3, df_smaller4])
census_final.head()

,PUMA,ST,WAGP,AGEP,SCHL,RAC1P
0,2300,1,36000.0,47,10.0,1
1,1700,1,30000.0,59,9.0,1
2,1700,1,0.0,57,9.0,1
3,1700,1,28000.0,30,11.0,1
4,1700,1,19000.0,50,6.0,1


In [7]:
# Saves census dataframe into a pickle
file_path = 'census_data.pickle'
census_final = pd.read_pickle(file_path)

In [8]:
def get_state (row):
    """
    Takes in a dataframe row and returns the state as a string
    """
    dict_state = { 1: 'Alabama', 2: 'Alaska', 4: 'Arizona', 5: 'Arkansas', 6: 'California',
                   8: 'Colorado', 9: 'Connecticut', 10: 'Delaware', 11: 'District Of Columbia', 
                   12: 'Florida', 13: 'Georgia', 15: 'Hawaii', 16: 'Idaho', 17: 'Illinois', 
                   18: 'Indiana', 19: 'Iowa', 20: 'Kansas', 21: 'Kentucky', 22: 'Louisiana',
                   23: 'Maine', 24: 'Maryland', 25: 'Massachusetts', 26: 'Michigan', 27: 'Minnesota',
                   28: 'Mississippi', 29: 'Missouri', 30: 'Montana', 31: 'Nebraska', 32: 'Nevada',
                   33: 'New Hampshire', 34: 'New Jersey', 35: 'New Mexico', 36: 'New York', 
                   37: 'North Carolina', 38: 'North Dakota', 39: 'Ohio', 40: 'Oklahoma', 41: 'Oregon',
                   42: 'Pennsylvania', 44: 'Rhode Island', 45: 'South Carolina', 46: 'South Dakota', 
                   47: 'Tennessee', 48: 'Texas', 49: 'Utah', 50: 'Vermont', 51: 'Virginia', 53: 'Washington',
                   54: 'West Virginia', 55: 'Wisconsin', 56: 'Wyoming', 72: 'Puerto Rico'}
    state = row['ST']
    return dict_state[state]

In [9]:
def get_race (row):
    """
    Takes in a dataframe row and returns the race as a string
    """
    dict_race = { 1: 'White', 2: 'Black', 3: 'American Indian', 4: 'Alaska Native', 5: 'American Indian',
                  6: 'Asian', 7: 'Native Hawaiian', 8: 'Other', 9: 'Two or More'
                }
    race = row['RAC1P']
    return dict_race[race]

In [22]:
def get_dummy(row, varname, dummy):
    """
    Takes in a dataframe row, the name of the column (varname), and dummy
    and returns 1 if the value matches dummy and 0 if the values does not match
    """
    name = row[varname]
    if name in dummy:
        return 1
    else:
        return 0

In [11]:
# Modifies the Race and State variables.  
census_final['Race'] = census_final.apply(get_race, axis = 1)
census_final['State'] = census_final.apply(get_state, axis = 1)

In [14]:
# Makes a variable that is the combination of the PUMA and state
census_final['PUMA_State'] = census_final['PUMA'].astype(str) +' '+ census_final['State']

The following code adds MSA (Metropolitan Statistical Area) data into the census data.  I had originally envisioned using MSA data in the final models but it turns out it is not trivial to match the MSA labels for the census data with the MSA labels for the murders data.

Ended up only using average state data from the census in our model.  However, MSA data is there so we can add MSA data to the models in the future.

In [15]:
# Reads a csv with MSA data into a dataframe
msa_df = pd.read_csv('MSA.csv')

In [16]:
msa_df.head()

,MSA Code,MSA Title,State FIPS Code,State Name,2000 PUMA Code,MSA 2010 Population,2000 PUMA Est. 2010 Population,Intersection Est. 2010 Population,Percent MSA Population,Percent PUMA Population
0,10180,"Abilene, TX",48,Texas,2700,"165,252","161,875","13,544",8.20,8.37
1,10180,"Abilene, TX",48,Texas,2800,"165,252","116,673","20,202",12.22,17.32
2,10180,"Abilene, TX",48,Texas,2900,"165,252","131,506","131,506",79.58,100.00
3,10380,"Aguadilla-Isabela, PR",72,Puerto Rico,100,"339,441","142,949","142,949",42.11,100.00
4,10380,"Aguadilla-Isabela, PR",72,Puerto Rico,200,"339,441","148,635","45,604",13.44,30.68


In [17]:
# Combines PUMA and State variables
msa_df['Puma_State'] = msa_df['2000 PUMA Code'].astype(str) + ' ' + msa_df['State Name']

In [18]:
# Merges the census data and msa data on the combined PUMA and State variable
census_msa_df = census_final.merge(msa_df, how='left', left_on = 'PUMA_State', right_on = 'Puma_State')

In [23]:
# Gets dummy variables from the Race variable
census_msa_df = census_final.merge(msa_df, how='left', left_on = 'PUMA_State', right_on = 'Puma_State')
census_msa_df['White'] = census_msa_df.apply(get_dummy, varname = 'Race', dummy = 'White', axis = 1)
census_msa_df['Asian'] = census_msa_df.apply(get_dummy, varname = 'Race', dummy = 'Asian', axis = 1)
census_msa_df['Native American'] = census_msa_df.apply(get_dummy, varname = 'Race', 
                                                       dummy = ['American Indian', 'Alaska Native', 'Native Hawaiian'], axis = 1)

In [ ]:
# Creates a dataframe that has the average income, level of schooling, and percentage of population that are black, white, and Native American
# by state as a dataframe.  Basically gives general demographic information in each state.
state_demo = census_msa_df.groupby('State').mean().reset_index()[['State', 'WAGP', 'SCHL', 'Black', 'White', 'Native American']]

In [29]:
# Saves the state_demo dataframe into a pickle
with open('state_demo.pickle', 'wb') as to_write:
    pickle.dump(state_demo, to_write)